In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

driver = webdriver.Chrome()
driver.get('https://vestibular.brasilescola.uol.com.br/banco-de-redacoes')

# Get the essay theme table
theme_table = driver.find_element(By.ID, 'table-temas')

In [3]:
table_rows = theme_table.find_elements(By.TAG_NAME, 'tr')[1:]

themes = []
for row in table_rows:
    a_element = row.find_elements(By.TAG_NAME, 'td')[0].find_element(By.TAG_NAME, 'a')
    link = a_element.get_property('href')
    theme = a_element.text
    date = row.find_elements(By.TAG_NAME, 'td')[1].text
    month = date.split('/')[0]
    year = date.split('/')[1]
    themes.append([theme, link, month, year])

df = pd.DataFrame(themes, columns=['Theme', 'Link', 'Month', 'Year'])
df.to_csv('data/links.csv', index=False)

In [8]:
import hashlib
from pathlib import Path

output_dir = Path('data/themes')

output_dir.mkdir(parents=True, exist_ok=True)

links = [df['Link'][0]]
for link in links:
    driver.get(link)
    theme_paragraph = driver.find_element(By.CLASS_NAME, 'texto-conteudo').find_element(By.TAG_NAME, 'p').text
    table_corrected = driver.find_element(By.ID, 'redacoes_corrigidas')
    essays_corrected = table_corrected.find_elements(By.TAG_NAME, 'tr')[1:]
    essays_links = []
    for essay, n in zip(essays_corrected, range(len(essays_corrected))):
        a_element = essay.find_elements(By.TAG_NAME, 'td')[0].find_element(By.TAG_NAME, 'a')
        link = a_element.get_property('href')
        number = n
        essays_links.append([theme_paragraph, n, link])
    theme_hash = hashlib.sha256(link.encode()).hexdigest()
    df_theme = pd.DataFrame(essays_links, columns=['Proposition', 'Id', 'Link'])
    df_theme.to_csv(f'{output_dir}/{theme_hash}.csv', index=False)

In [9]:
link = df_theme['Link'][0]
driver.get(link)
essay_raw = driver.find_element(By.CLASS_NAME, 'area-redacao-corrigida').find_element(By.TAG_NAME, 'p')


In [23]:
import re

def remove_comments(all_red: list):
    clean = []
    for item in all_red:
        new_item = re.sub("\(.*\)", "", item.text).strip()
        if new_item:
            clean.append(new_item)
    return clean

In [24]:
all_red = essay_raw.find_elements(By.TAG_NAME, "span")
clean = remove_comments(all_red)

In [ ]:
def clean_essay(essay_raw: str):
    return re.sub("\(.*\)", "", essay_raw)

In [33]:
essay_clean = re.sub("\(.*?\)", "", essay_raw.text)


'Segundo, a Constituição Federal de 1988, Art 27 é dever da família, da sociedade, assegurar os direitos básicos da criança e do adolescente, como à saúde, à segurança. No entanto, as obrigações mencionadas na Carta Magna não são colocadas em prática, visto que há o descaso familiar em controlar o tempo de uso de aparelhos tecnológicos usados pela criança. Assim, é necessário discutir o problema do aumento da ansiedade infantil por causa da tecnologia e suas possíveis causas como a omissão da família e a influência das redes sociais. Diante desse cenário, um dos fatores desse impasse é a negligência familiar em gerenciar o tempo de exposição da criança a tecnologia. Nesse sentido, o Dr. Drauzio Varella afirma que o excesso de contato com aparelhos tecnológicos prejudica o sono dos meninos, que é muito importante para o seu desenvolvimento. Dessa forma, os responsáveis devem estabelecer um limite para acessar esses dispositivos, uma vez que esses mecanismos podem impactar o funcionament

In [30]:
essay_raw.text

'Segundo, (Sem vírgula) a Constituição Federal de 1988, Art 27 é dever da família, da sociedade, assegurar os direitos básicos da criança e do adolescente, como à saúde, à segurança. No entanto, as obrigações mencionadas na Carta Magna não são colocadas em prática, visto que há o descaso familiar em controlar o tempo de uso de aparelhos tecnológicos usados pela criança. Assim, é necessário discutir o problema do aumento da ansiedade infantil por causa da tecnologia e suas possíveis causas (Vírgula) como a omissão da família e a influência das redes sociais. (Muito bem. Formulou a tese)\n\nDiante desse cenário, um dos fatores desse impasse é a negligência familiar em gerenciar o tempo de exposição da criança a tecnologia. Nesse sentido, o Dr. Drauzio Varella afirma que o excesso de contato com aparelhos tecnológicos prejudica o sono dos meninos, que é muito importante para o seu desenvolvimento. Dessa forma, os responsáveis devem estabelecer um limite para acessar esses dispositivos, um